In [6]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
from langchain_community.embeddings import SentenceTransformerEmbeddings
import os
from langchain_community.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from datasets import load_dataset
import pandas as pd

In [10]:
loader = DirectoryLoader('data/', glob="**/*.pdf", show_progress=True, loader_cls=UnstructuredFileLoader)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = SentenceTransformerEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

100%|██████████| 10/10 [00:07<00:00,  1.43it/s]
Created a chunk of size 1147, which is longer than the specified 500
Created a chunk of size 1570, which is longer than the specified 500
Created a chunk of size 639, which is longer than the specified 500
Created a chunk of size 610, which is longer than the specified 500
Created a chunk of size 1008, which is longer than the specified 500
Created a chunk of size 545, which is longer than the specified 500
Created a chunk of size 536, which is longer than the specified 500
Created a chunk of size 583, which is longer than the specified 500
Created a chunk of size 1162, which is longer than the specified 500
Created a chunk of size 607, which is longer than the specified 500
Created a chunk of size 1206, which is longer than the specified 500
Created a chunk of size 697, which is longer than the specified 500
Created a chunk of size 734, which is longer than the specified 500
Created a chunk of size 916, which is longer than the specified

In [11]:
db = Chroma.from_documents(docs, embeddings)

In [ ]:
dataset = load_dataset('csv', data_files=r'C:\Users\adrianhf\Documents\test\Master\data\synthetic_data\question_with_answers.csv', split="train[:10]")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("RuterNorway/Llama-2-13b-chat-norwegian")
model = AutoModelForCausalLM.from_pretrained("RuterNorway/Llama-2-13b-chat-norwegian")

In [ ]:
answers_from_model = []
for i in range(10):
    query = dataset["Question"][i]
    found_docs = db.similarity_search(query)
    context = found_docs[0].page_content
    input = f"Spørsmål: {query} context: {context}"
    instruction = "Svar på spørsmålet basert på det som står i 'context'"
    prompt_template=f'''### Instruction: {instruction}
    ### Input: {input}
    ### Response:
    '''
    print("\n\n*** Generate:")
    inputs = tokenizer(prompt_template, return_tensors="pt")

    out = model.generate(**inputs, max_new_tokens=200)
    print(tokenizer.decode(out[0], skip_special_tokens=True))

    # Pipeline prompting
    print("\n\n*** Pipeline:\n\n")
    pipe = pipeline(
        "text-generation",
        model=model,
        do_sample=True,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )
    print(pipe(prompt_template)[0]['generated_text'][len(prompt_template):])
    answers_from_model.append(pipe(prompt_template)[0]['generated_text'][len(prompt_template):])

In [13]:
# Convert the list to a pandas DataFrame
df = pd.DataFrame(answers_from_model, columns=['Text'])

# Specify the file path
file_path = "C:\\Users\\adrianhf\\Documents\\test\\Master\\data\\Results\\Chroma_answers_from_model.csv"

# Write the DataFrame to a CSV file
df.to_csv(file_path, index=False)

For hver manglende oppstillingsplass betaler tiltakshaver til enhver tid gjeldende frikjøpsbeløp til Kristiansund kommune. Frikjøpsbeløpet er for 2020 NOK 360 000 per oppstillingsplass for bil og NOK 41 000 per oppstillings- plass for sykkel. Beløpet kan prisjusteres årlig som del av budsjettbehandlingen. Innbetalte midler plasseres i kommunalt fond som gis betegnelsen ”Fond til parkeringsformål for Kristiansund kommune”. Fondets midler med renter disponeres av Kristiansund kommune. Fondets midler skal kun brukes til grunnerverv og opparbeid- else for offentlig parkeringsanlegg for bil og/eller sykkel, med tilhørende infrastruktur.
